In [34]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device, torch.version.cuda)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(torch_dtype)


# dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
# sample = dataset[0]["audio"]

# result = pipe(sample, return_timestamps = True)
# print(result["text"])

# result = pipe("audio.mp3")
# result = pipe(["audio_1.mp3", "audio_2.mp3"], batch_size=2)

# generate_kwargs = {
#     "max_new_tokens": 448,
#     "num_beams": 1,
#     "condition_on_prev_tokens": False,
#     "compression_ratio_threshold": 1.35,  # zlib compression ratio threshold (in token space)
#     "temperature": (0.0, 0.2, 0.4, 0.6, 0.8, 1.0),
#     "logprob_threshold": -1.0,
#     "no_speech_threshold": 0.6,
#     "return_timestamps": True,
# }

# result = pipe(sample, generate_kwargs=generate_kwargs)

# result = pipe(sample, generate_kwargs={"language": "english"})

# result = pipe(sample, return_timestamps=True)
# print(result["chunks"])


cuda:0 11.8
torch.float16


In [35]:
model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, 
    # low_cpu_mem_usage=True, 
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

# pipe = pipeline(
#     "automatic-speech-recognition",
#     model=model,
#     tokenizer=processor.tokenizer,
#     feature_extractor=processor.feature_extractor,
#     chunk_length_s=30,
#     torch_dtype=torch_dtype,
#     device=device,
# )

## Quick test

In [36]:
# from datasets import load_dataset

# result = pipe(sample, return_timestamps = True)
# print(result["text"])


## Run on Audio

In [37]:
# result = pipe("audio/Skip the dishes.mp3",return_timestamps= True)
# print(result)
    
# # save to txt
# import time
# import json

# with open("transcripts/result_" +str(time.time()) + ".json", 'w') as f:
#     f.write(json.dumps(result))

# torch.cuda.empty_cache()

## Alternate code until fix is applied

In [38]:
import json

def process_audio(audio_path):
    # Load and preprocess audio
    audio = processor(audio_path, return_tensors="pt", sampling_rate=16000)
    audio = audio.to(device, torch_dtype)
    
    # Generate with timestamps and segments
    output = model.generate(
        **audio,
        return_timestamps=True,
        return_segments=True
    )
    
    # Decode and process results
    result = processor.batch_decode(
        output['sequences'], 
        skip_special_tokens=True, 
        output_offsets=True
    )
    
    # Add correct timestamps from segments
    for i in range(len(result[0]['offsets'])):
        result[0]['offsets'][i]['timestamp'] = (
            output['segments'][0][i]['start'].item(),
            output['segments'][0][i]['end'].item()
        )
    
    return result[0]  # Return first result since we're processing single audio file

# Process the audio file
result = process_audio("audio/Skip the dishes.mp3")

# Save to file
with open(f"transcripts/result_{str(time.time())}.json", 'w') as f:
    json.dump(result, f, indent=4)

# Clear CUDA cache
torch.cuda.empty_cache()

ValueError: could not convert string to float: 'audio/Skip the dishes.mp3'

In [39]:
!pip install librosa

  Using cached librosa-0.10.2.post1-py3-none-any.whl.metadata (8.6 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached scipy-1.15.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached numba-0.61.0-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached soundfile-0.13.0-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.5.0.post1-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
  Using cached msgpack-1.1.0-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
  Using cached llvmlite-0.44.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl.metadata (1.6 kB)
  Using cache